<font color="#D31525"><h3 align="left">Detección de fraude en transacciones financieras</h3></font>
<font color="#2C3E50"><h3 align="left">GENERACIÓN DE DATASET APLICANDO LA TECNICA DE BALANCEO DE CLASES UNDERSAMPLING</h3></font>
<font color="#2C3E50"><h3 align="left">GENERACION FICHERO TRAIN-TEST Y EVALUACION USAN PCA</h3></font>

## Importar librerias
En esta primera parte del código, se realizan las llamadas a las librerías que se utilizarán en el Notebook:

In [1]:
# Paquetes de manipulación de datos
import pandas as pd
import numpy as np
import boto3

# Paquetes de visualización
import matplotlib.pyplot as plt
import seaborn as sns

# Paquete de manipulación de fechas
import datetime as dt

# Paquete para hacer el PCA.
from sklearn.decomposition import PCA


## Importar DataSet
Una vez que hemos limpiado el dataset, realizamos el estudio descriptivo y discovery de los datos. Para ello, importamos el dataset limpio:

In [2]:
s3 = boto3.client("s3")

# Seleccionamos el bucket con el que vamos a trabajar
BUCKET_NAME = 'tfmfraud'

In [3]:
# Descargamos el fichero del bucket de s3 a la máquina EC2 para poder trabajar con él.
s3.download_file(Bucket = BUCKET_NAME, Key = 'df_new_var.csv',Filename = '/tmp/df_new_var.csv')

In [4]:
#Leemos el fichero y lo metemos en un dataframe.
df = pd.read_csv('/tmp/df_new_var.csv', dtype={'rank':'category'})

In [5]:
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud,day,week,isFlaggedFraud_New,ind_merchant,balanceOrig,balanceDest,hours_day,amount_category
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0.0,0.0,1,1,0,1,9839.64,0.0,1,"(-0.001, 200000.0]"
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0.0,0.0,1,1,0,1,1864.28,0.0,1,"(-0.001, 200000.0]"
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1.0,0.0,1,1,0,0,181.00,0.0,1,"(-0.001, 200000.0]"
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1.0,0.0,1,1,0,0,181.00,-21182.0,1,"(-0.001, 200000.0]"
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0.0,0.0,1,1,0,1,11668.14,0.0,1,"(-0.001, 200000.0]"


In [6]:
# Eliminar el fichero de la ruta tmp de la máquina EC2 para no ocupar espacio.
!rm /tmp/df_new_var.csv

In [7]:
df.shape

(6362620, 19)

In [9]:
pd.value_counts(df['isFraud'])

0.0    6354407
1.0       8213
Name: isFraud, dtype: int64

Miramos proporcion CUSTOMER/ MERCHANT tenemos en nuestro dataset.

In [10]:
pd.value_counts(df['ind_merchant'])

0    4211125
1    2151495
Name: ind_merchant, dtype: int64

In [9]:
pd.crosstab(index=df['ind_merchant'],columns=df['isFraud'], margins=True)

isFraud,0.0,1.0,All
ind_merchant,,,
0,4202912,8213,4211125
1,2151495,0,2151495
All,6354407,8213,6362620


In [10]:
pd.value_counts(df['type'])

CASH_OUT    2237500
PAYMENT     2151495
CASH_IN     1399284
TRANSFER     532909
DEBIT         41432
Name: type, dtype: int64

In [12]:
pd.crosstab(index=df['type'],columns=df['isFraud'], margins=True)

isFraud,0.0,1.0,All
type,,,
CASH_IN,1399284,0,1399284
CASH_OUT,2233384,4116,2237500
DEBIT,41432,0,41432
PAYMENT,2151495,0,2151495
TRANSFER,528812,4097,532909
All,6354407,8213,6362620


In [11]:
pd.crosstab(index=df['type'],columns=df['ind_merchant'], margins=True)

ind_merchant,0,1,All
type,,,
CASH_IN,1399284,0,1399284
CASH_OUT,2237500,0,2237500
DEBIT,41432,0,41432
PAYMENT,0,2151495,2151495
TRANSFER,532909,0,532909
All,4211125,2151495,6362620


Limpiamos el dataset para ir balanceando la muestra:

Eliminamos aquellos registros que cumplen: 

1. Tanto el origen como el destinatario son clientes, no comercios
2. El valor de la variable amount es mayor que 0 
3. Tanto el balance de origen como de destino es 0
4. No salen marcados en nuestro dataset como movimientos fraudulentos.

In [8]:
# Tabla de contingencia: Customer
pd.crosstab(index=((df['oldbalanceOrg'] == 0) &
                  (df['newbalanceOrig'] == 0) &
                  (df['oldbalanceDest'] == 0) &
                  (df['newbalanceDest'] == 0) &
                  (df["ind_merchant"]== 0) &
                  (df['amount'] > 0) & (df['isFraud'] == 0)),columns=df['type'], margins=True)

type,CASH_IN,CASH_OUT,DEBIT,PAYMENT,TRANSFER,All
row_0,,,,,,
False,1399284,2237481,41432,2151495,532904,6362596
True,0,19,0,0,5,24
All,1399284,2237500,41432,2151495,532909,6362620


Eliminar estos registros queda justificado en la descripción del dataset, donde podemos leer
*"las categorías PAYMENT, TRANSFER y CASH_OUT implican variación en 
los balances de las cuentas origen y destino."* Puesto que estos casos no cumplen con lo definido, se considerarán outliers 
y se eliminaran del dataframe ya que solo generan ruido.

In [11]:
df1 = df.drop(df[(df['oldbalanceOrg'] == 0) &
               (df['newbalanceOrig'] == 0) &
               (df['oldbalanceDest'] == 0) &
               (df['newbalanceDest'] == 0) &
               (df["ind_merchant"]== 0) &
               (df['amount'] > 0) & 
               (df['isFraud'] == 0)].index)

In [12]:
df1.shape

(6362596, 19)

Eliminamos aquellos registros que cumplen: 

1. El destinatario son comercios
2. El valor de la variable amount es mayor que 0 
3. El balance de origen es 0 (En este dataset los balances de los comercios destinatiarios siempre es 0)
4. No salen marcados en nuestro dataset como movimientos fraudulentos.

In [11]:
# Tabla de contingencia: Merchant
pd.crosstab(index=((df1['oldbalanceOrg'] == 0) &
                  (df1['newbalanceOrig'] == 0) &
                  (df1["ind_merchant"]== 1) &
                  (df1['amount'] > 0) & (df1['isFraud'] == 0)),columns=df1['type'], margins=True)

type,CASH_IN,CASH_OUT,DEBIT,PAYMENT,TRANSFER,All
row_0,,,,,,
False,1399284,2237481,41432,1377250,532904,5588351
True,0,0,0,774245,0,774245
All,1399284,2237481,41432,2151495,532904,6362596


In [13]:
 df2 = df.drop(df1[(df1['oldbalanceOrg'] == 0) &
               (df1['newbalanceOrig'] == 0) &
               (df1["ind_merchant"]== 1) &
               (df1['amount'] > 0) & 
               (df1['isFraud'] == 0)].index)

In [14]:
df2.shape

(5588375, 19)

In [15]:
pd.value_counts(df2['ind_merchant'])

0    4211125
1    1377250
Name: ind_merchant, dtype: int64

Miramos la distribución de operaciones fraudeulentas por tipo de transaccion

In [18]:
pd.crosstab(index=df2['type'],columns=df2['isFraud'], margins=True)

isFraud,0.0,1.0,All
type,,,
CASH_IN,1399284,0,1399284
CASH_OUT,2233384,4116,2237500
DEBIT,41432,0,41432
PAYMENT,1377250,0,1377250
TRANSFER,528812,4097,532909
All,5580162,8213,5588375


Del resultado vemos, que solo hay operaciones fraudulentas en 2 categorías de todas, por tanto, eliminamos las categorias que no tienen transacciones fraudulentas, porque distorsionarían los resultados. Ya que en estos casos no se podría predecir nunca fraude puesto que todos los casos que se entrenarían serían no fraude.

In [11]:
df3 = df2[df2.type.isin(["CASH_OUT", "TRANSFER"])]

In [13]:
df3.shape

(2770409, 19)

In [14]:
pd.crosstab(index=df3['type'],columns=df3['isFraud'], margins=True)

isFraud,0.0,1.0,All
type,,,
CASH_OUT,2233384,4116,2237500
TRANSFER,528812,4097,532909
All,2762196,8213,2770409


In [15]:
pd.crosstab(index=df3['type'],columns=df3['ind_merchant'], margins=True)

ind_merchant,0,All
type,,
CASH_OUT,2237500,2237500
TRANSFER,532909,532909
All,2770409,2770409


Eliminamos los dataframes df y df2 para liberar espacio.

In [12]:
del df
del df2

#### Del dataframe df3 que tiene 2,7M de registros, vamos a sacar 2 dataset diferentes.
1. Dataset para Train-test. Este dataset va a tener una proporcion No fraude - Fraude del 90%-10% (66.717 transacciones no fraudulentas y 7.413 de transacciones fraudulentas)
2. Dataset para evaluacion.Este dataset va a tener un total de 80.000 registros cumpliendo una proporcion No fraude - Fraude del 99%-1%(79.200 transacciones no fraudulentas y 800 de transacciones fraudulentas)

In [13]:
fraude = df3.loc[df3['isFraud'] == 1]
noFraude = df3.loc[df3['isFraud'] == 0]

## **FRAUDE**

In [14]:
evalFr = fraude.sample(800)
trainFr = fraude[~fraude.index.isin(evalFr.index)]

In [15]:
print(evalFr.shape)
print(trainFr.shape)

(800, 19)
(7413, 19)


## **NO FRAUDE**

In [16]:
evalNoFr = noFraude.sample(79200)
trainNoFr = noFraude[~noFraude.index.isin(evalNoFr.index)].sample(66717)

In [17]:
print(evalNoFr.shape)
print(trainNoFr.shape)

(79200, 19)
(66717, 19)


Union de los dataframes de Evaluacion de Evaluacion y Train-Test

In [18]:
evaluacion = pd.concat([evalFr,evalNoFr])
trainTest =  pd.concat([trainFr,trainNoFr])

In [19]:
print(evaluacion.shape)
print(trainTest.shape)

(80000, 19)
(74130, 19)


In [27]:
pd.crosstab(index=trainTest['type'],columns=trainTest['isFraud'], margins=True)

isFraud,0.0,1.0,All
type,,,
CASH_OUT,53800,3735,57535
TRANSFER,12917,3678,16595
All,66717,7413,74130


In [20]:
pd.crosstab(index=evaluacion['type'],columns=evaluacion['isFraud'], margins=True)

isFraud,0.0,1.0,All
type,,,
CASH_OUT,63907,403,64310
TRANSFER,15293,397,15690
All,79200,800,80000


Eliminamos los dataframes df3 para liberar memoria.

In [21]:
del df3

Guardamos los ficheros tras eliminar registros de la clase mayoritaria y poder usarlos cuando sean necesarios.

In [35]:
trainTest.to_csv('/tmp/train_test_under.csv', index = False)
evaluacion.to_csv('/tmp/evaluacion_under.csv', index = False)

In [36]:
s3.upload_file(Bucket = BUCKET_NAME, Key = 'train_test_under.csv', Filename = '/tmp/train_test_under.csv')
s3.upload_file(Bucket = BUCKET_NAME, Key = 'evaluacion_under.csv', Filename = '/tmp/evaluacion_under.csv')

In [37]:
# Eliminar el fichero de la ruta tmp de la máquina EC2 para no ocupar espacio.
!rm /tmp/train_test_under.csv
!rm /tmp/evaluacion_under.csv

Leemos el fichero de train_test_under para aplicar PCA. De esta tenemos un checkpoint para no tener que rehacer todo el trabajo previo

In [170]:
s3 = boto3.client("s3")

# Seleccionamos el bucket con el que vamos a trabajar
BUCKET_NAME = 'tfmfraud'

In [171]:
# Descargamos el fichero del bucket de s3 a la máquina EC2 para poder trabajar con él.
s3.download_file(Bucket = BUCKET_NAME, Key = 'train_test_under.csv',Filename = '/tmp/train_test_under.csv')

In [172]:
#Leemos el fichero y lo metemos en un dataframe.
trainTest = pd.read_csv('/tmp/train_test_under.csv', dtype={'rank':'category'})

In [173]:
trainTest.shape

(74130, 18)

In [174]:
trainTest.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud,day,week,isFlaggedFraud_New,ind_merchant,balanceOrig,balanceDest,amount_category
0,1,TRANSFER,181.0,C1305486145,181.0,0.0,C553264065,0.0,0.0,1.0,0.0,1,1,0,0,181.0,0.0,"(-0.001, 13389.57]"
1,1,CASH_OUT,181.0,C840083671,181.0,0.0,C38997010,21182.0,0.0,1.0,0.0,1,1,0,0,181.0,-21182.0,"(-0.001, 13389.57]"
2,1,TRANSFER,2806.0,C1420196421,2806.0,0.0,C972765878,0.0,0.0,1.0,0.0,1,1,0,0,2806.0,0.0,"(-0.001, 13389.57]"
3,1,CASH_OUT,2806.0,C2101527076,2806.0,0.0,C1007251739,26202.0,0.0,1.0,0.0,1,1,0,0,2806.0,-26202.0,"(-0.001, 13389.57]"
4,1,TRANSFER,20128.0,C137533655,20128.0,0.0,C1848415041,0.0,0.0,1.0,0.0,1,1,0,0,20128.0,0.0,"(13389.57, 74871.94]"


In [193]:
pd.crosstab(index=trainTest['type'],columns=trainTest['isFraud'], margins=True)

isFraud,0.0,1.0,All
type,,,
CASH_OUT,53880,3721,57601
TRANSFER,12837,3692,16529
All,66717,7413,74130


In [175]:
# Eliminar el fichero de la ruta tmp de la máquina EC2 para no ocupar espacio.
!rm /tmp/df_new_var.csv

rm: cannot remove ‘/tmp/df_new_var.csv’: No such file or directory


Preparamos el dataframe *trainTest* para aplicar un PCA
Eliminamos las columnas: isFraud, nameOrig, nameDest

In [176]:
mydata2 = trainTest.drop(['isFraud','nameOrig', 'nameDest', 'amount_category'],axis = True)
fraud = pd.DataFrame(trainTest['isFraud'])

In [177]:
mydata2.shape

(74130, 14)

Generamos una variable dummy del campos type

In [178]:
mydata2Dummies = pd.get_dummies(mydata2, drop_first=True)

In [179]:
mydata2Dummies.shape

(74130, 14)

In [180]:
mydata2Dummies.head()

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFlaggedFraud,day,week,isFlaggedFraud_New,ind_merchant,balanceOrig,balanceDest,type_TRANSFER
0,1,181.0,181.0,0.0,0.0,0.0,0.0,1,1,0,0,181.0,0.0,1
1,1,181.0,181.0,0.0,21182.0,0.0,0.0,1,1,0,0,181.0,-21182.0,0
2,1,2806.0,2806.0,0.0,0.0,0.0,0.0,1,1,0,0,2806.0,0.0,1
3,1,2806.0,2806.0,0.0,26202.0,0.0,0.0,1,1,0,0,2806.0,-26202.0,0
4,1,20128.0,20128.0,0.0,0.0,0.0,0.0,1,1,0,0,20128.0,0.0,1


In [181]:
fraud.head()

,isFraud
0,1.0
1,1.0
2,1.0
3,1.0
4,1.0


In [182]:
fraud.shape

(74130, 1)

* **PCA**

In [183]:
pca = PCA()
X_pca = pca.fit_transform(mydata2Dummies.values)

In [184]:
pca.explained_variance_ratio_.cumsum()

array([0.8560182 , 0.94824483, 0.98835843, 0.99620646, 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        ])

Viendo que con 4 variables explicamos el 99,6% de la varianza, entonces aplicaremos un PCA con 4 componentes principales. El motivo de aplicar el procedimiento de Componentes Principales, es para poder enfrentar modelos explicativos Vs modelos no explictivos.

Eliminamos los dataframes mydata2 y la lista X_pca para liberar memoria.

In [185]:
del mydata2
del X_pca

In [186]:
pca = PCA(4)
dfX_pca4 = pd.DataFrame(pca.fit_transform(mydata2Dummies.values))

In [187]:
pca.explained_variance_ratio_.cumsum()

array([0.8560182 , 0.94824483, 0.98835843, 0.99620646])

In [188]:
dfX_pca4.rename(columns={0: 'PC0'
                    , 1: 'PC1'
                    , 2: 'PC2'
                    , 3: 'PC3'}, inplace=True)
dfX_pca4.head()

,PC0,PC1,PC2,PC3
0,-2.568416e+06,-418753.356792,-26107.093843,36123.614588
1,-2.556271e+06,-435297.382974,-7558.987926,28869.723793
2,-2.568229e+06,-415170.264690,-23825.616824,34557.111152
3,-2.553205e+06,-435635.120416,-881.727142,25584.094128
4,-2.566995e+06,-391525.951775,-8770.475899,24219.978765


In [189]:
print(dfX_pca4.shape)
print(fraud.shape)

(74130, 4)
(74130, 1)


Unimos el dataframe de los campos del PCA con su valor correspondiente del campo isFraud y lo guardaremos en un dataset

In [190]:
trainTestPca = pd.merge(dfX_pca4,fraud, right_index=True, left_index=True)

In [191]:
trainTestPca.shape

(74130, 5)

In [192]:
trainTestPca.head(10)

,PC0,PC1,PC2,PC3,isFraud
0,-2.568416e+06,-4.187534e+05,-2.610709e+04,3.612361e+04,1.0
1,-2.556271e+06,-4.352974e+05,-7.558988e+03,2.886972e+04,1.0
2,-2.568229e+06,-4.151703e+05,-2.382562e+04,3.455711e+04,1.0
3,-2.553205e+06,-4.356351e+05,-8.817271e+02,2.558409e+04,1.0
4,-2.566995e+06,-3.915260e+05,-8.770476e+03,2.421998e+04,1.0
5,-2.553321e+06,-3.886462e+05,-1.267133e+04,2.611453e+04,1.0
6,5.170450e+06,5.756482e+06,-7.228772e+06,3.002444e+06,1.0
7,-2.477455e+06,1.324379e+06,1.083805e+06,-7.259620e+05,1.0
8,-4.483534e+05,2.889566e+06,-8.063123e+05,8.751143e+04,1.0
9,-2.565931e+06,-3.711390e+05,4.210589e+03,1.530694e+04,1.0


Chequeamos que el numero de fraude y no fraude son correctos.

In [1]:
print(pd.value_counts(trainTestPca['isFraud']))

NameError: name 'pd' is not defined

**Guardamos el nuevo dataset**  
Guardamos el fichero usando en nuestro bucket de s3 usando la libreria **boto3**

In [197]:
trainTestPca.to_csv('/tmp/train_test_under_pca.csv', index = False)

In [198]:
s3.upload_file(Bucket = BUCKET_NAME, Key = 'train_test_under_pca.csv', Filename = '/tmp/train_test_under_pca.csv')

In [199]:
# Eliminar el fichero de la ruta tmp de la máquina EC2 para no ocupar espacio.
!rm /tmp/train_test_under_pca.csv

Ahora se aplica el PCA al dataframe de Evaluacion *evaluacion*.
Leemos el fichero que hemos guardado en el paso anterior. De esta tenemos un checkpoint para no tener que rehacer todo el trabajo previo.

In [200]:
s3 = boto3.client("s3")

# Seleccionamos el bucket con el que vamos a trabajar
BUCKET_NAME = 'tfmfraud'

In [201]:
# Descargamos el fichero del bucket de s3 a la máquina EC2 para poder trabajar con él.
s3.download_file(Bucket = BUCKET_NAME, Key = 'evaluacion_under.csv',Filename = '/tmp/evaluacion_under.csv')

In [202]:
#Leemos el fichero y lo metemos en un dataframe.
evaluacion = pd.read_csv('/tmp/evaluacion_under.csv', dtype={'rank':'category'})

In [203]:
evaluacion.shape

(80000, 18)

In [204]:
evaluacion.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud,day,week,isFlaggedFraud_New,ind_merchant,balanceOrig,balanceDest,amount_category
0,178,TRANSFER,1796069.20,C913178524,1796069.20,0.0,C553554525,0.00,0.00,1.0,0.0,8,2,1,0,1796069.20,0.00,"(208721.478, 92445516.64]"
1,337,CASH_OUT,88748.63,C1502031409,88748.63,0.0,C997271871,664794.37,753543.00,1.0,0.0,15,2,0,0,88748.63,88748.63,"(74871.94, 208721.478]"
2,523,TRANSFER,248799.49,C315569976,248799.49,0.0,C1189969071,0.00,0.00,1.0,0.0,22,3,1,0,248799.49,0.00,"(208721.478, 92445516.64]"
3,94,CASH_OUT,1454592.61,C824268591,1454592.61,0.0,C819390198,264042.92,1718635.53,1.0,0.0,4,1,1,0,1454592.61,1454592.61,"(208721.478, 92445516.64]"
4,125,TRANSFER,144065.75,C699771361,144065.75,0.0,C1114367477,0.00,0.00,1.0,0.0,6,1,0,0,144065.75,0.00,"(74871.94, 208721.478]"


In [205]:
pd.crosstab(index=evaluacion['type'],columns=evaluacion['isFraud'], margins=True)

isFraud,0.0,1.0,All
type,,,
CASH_OUT,64146,395,64541
TRANSFER,15054,405,15459
All,79200,800,80000


In [206]:
# Eliminar el fichero de la ruta tmp de la máquina EC2 para no ocupar espacio.
!rm /tmp/evaluacion_under.csv

In [209]:
mydata2Eval = evaluacion.drop(['isFraud','nameOrig', 'nameDest', 'amount_category'],axis = True)
fraudEval = pd.DataFrame(evaluacion['isFraud'])

In [210]:
print(mydata2Eval.shape)
print(fraudEval.shape)

(80000, 14)
(80000, 1)


Generamos una variable dummy del campos type

In [211]:
mydata2EvalDummies = pd.get_dummies(mydata2Eval, drop_first=True)

In [212]:
dfeval_pca = pd.DataFrame(pca.transform(mydata2EvalDummies.values))

In [213]:
dfeval_pca.rename(columns={0: 'PC0'
                    , 1: 'PC1'
                    , 2: 'PC2'
                    , 3: 'PC3'}, inplace=True)
dfeval_pca.head()

,PC0,PC1,PC2,PC3
0,-2.440498e+06,2.032612e+06,1.534761e+06,-1.035596e+06
1,-1.555564e+06,-3.347021e+05,5.046703e+04,6.319293e+03
2,-2.550707e+06,-7.939223e+04,1.899757e+05,-1.122428e+05
3,-8.871237e+05,2.460477e+06,1.405758e+05,-3.504298e+05
4,-2.558167e+06,-2.223525e+05,9.894805e+04,-4.974151e+04


Unimos el dataframe de los campos del PCA con su valor correspondiente del campo isFraud y lo guardaremos en un dataset

In [214]:
print(dfeval_pca.shape)
print(fraudEval.shape)

(80000, 4)
(80000, 1)


In [215]:
evalPca = pd.merge(dfeval_pca,fraudEval, right_index=True, left_index=True)
evalPca.shape

(80000, 5)

Chequeamos que el numero de fraude y no fraude son correctos.

In [216]:
print(pd.value_counts(evalPca['isFraud']))

isFraud,0.0,1.0,All
PC0,,,
-2568428.815152692,0,1,1
-2568428.8151051104,0,1,1
-2568424.270954422,0,1,1
-2568404.504943538,0,1,1
-2568394.4904426937,0,1,1
...,...,...,...
187736437.77019206,1,0,1
228130918.8397613,1,0,1
329375485.2488518,0,1,1


**Guardamos el nuevo dataset**  
Guardamos el fichero usando en nuestro bucket de s3 usando la libreria **boto3**

In [217]:
evalPca.to_csv('/tmp/eval_under_pca.csv', index = False)

In [218]:
s3.upload_file(Bucket = BUCKET_NAME, Key = 'eval_under_pca.csv', Filename = '/tmp/eval_under_pca.csv')

In [58]:
# Eliminar el fichero de la ruta tmp de la máquina EC2 para no ocupar espacio.
!rm /tmp/eval_under_pca.csv